# Phase 0 Simulation - Stage 2: Full Scenario Testing

Runs all test cases T001-T010 (Scenarios A-G) with full integration:
- Mana distribution (75% deterministic + 25% Bloom VRF)
- Community detection (monthly Louvain clustering)
- Decay engine (progressive + anomaly-accelerated)
- Grace Period for returning users

**Success Criteria:**
- Sybil ROI(k) < 1/k
- TPR > 95%, FPR < 1%
- Gini: 0.2 - 0.3
- Community stability > 0.95

In [ ]:
import sys
sys.path.insert(0, '/Users/kunimitsu/Projects/Meguri_pre3')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
from pathlib import Path

from simulation.runner import SimulationRunner
from simulation.config import MeguriPhase0Config, TestCase
from simulation.metrics import generate_summary_report

np.random.seed(42)
plt.style.use('dark_background')
sns.set_palette('husl')

config = MeguriPhase0Config()
runner = SimulationRunner()

print(f"Loaded {len(config.test_cases)} test cases")
print("Ready to run Stage 2.")

## 1. Run All Test Cases (T001-T009)

T004 (N=100K) and T010 (variable k) are skipped in this quick run.

In [ ]:
# Default parameters (will be optimized in Stage 3)
KAPPA = 1.0
THETA = 0.3
BETA = 1.0
BLOOM_INTERVAL = 7

start = time.time()
all_results = runner.run_all_test_cases(
    kappa=KAPPA, theta=THETA, beta=BETA,
    bloom_interval=BLOOM_INTERVAL, verbose=True
)
elapsed = time.time() - start

print(f"\nTotal elapsed time: {elapsed:.1f}s")
print(f"Completed {len(all_results)} test cases")

## 2. Results Summary Table

In [ ]:
# Build summary DataFrame
summary_rows = []
for r in all_results:
    summary_rows.append({
        'Test ID': r.get('test_id', ''),
        'Scenario': r.get('scenario', ''),
        'N': r.get('network_size', ''),
        'k': r.get('attacker_count', 0),
        'Sybil ROI': r.get('sybil_roi', None),
        'ROI Threshold': r.get('sybil_roi_threshold', None),
        'ROI Pass': r.get('sybil_roi_pass', None),
        'TPR': r.get('tpr', None),
        'FPR': r.get('fpr', None),
        'TPR Pass': r.get('tpr_pass', None),
        'FPR Pass': r.get('fpr_pass', None),
        'Gini': r.get('gini', None),
        'Bloom Events': r.get('bloom_events', 0),
        'Stabilization (days)': r.get('stabilization_median_days', None),
    })

df_summary = pd.DataFrame(summary_rows)
df_summary

## 3. Sybil ROI Analysis

In [ ]:
# Filter scenario A tests (T001-T003)
scenario_a = df_summary[df_summary['Scenario'] == 'A'].copy()

if not scenario_a.empty:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Sybil ROI vs k
    ax1 = axes[0]
    k_vals = scenario_a['k'].values
    roi_vals = scenario_a['Sybil ROI'].values
    threshold_vals = scenario_a['ROI Threshold'].values
    
    ax1.plot(k_vals, roi_vals, 'o-', color='#ff6b6b', markersize=8, label='Actual ROI')
    ax1.plot(k_vals, threshold_vals, 's--', color='#51cf66', markersize=6, label='Target (1/k)')
    ax1.set_xlabel('Number of Attacker Nests (k)')
    ax1.set_ylabel('Sybil ROI')
    ax1.set_title('Sybil ROI vs Attacker Count (Scenario A)')
    ax1.legend()
    ax1.grid(alpha=0.3)
    ax1.set_yscale('log')
    
    # TPR and FPR for all scenarios
    ax2 = axes[1]
    test_ids = df_summary['Test ID'].values
    tpr_vals = df_summary['TPR'].values * 100
    fpr_vals = df_summary['FPR'].values * 100
    
    x = np.arange(len(test_ids))
    width = 0.35
    ax2.bar(x - width/2, tpr_vals, width, label='TPR (%)', color='#4ecdc4')
    ax2.bar(x + width/2, fpr_vals, width, label='FPR (%)', color='#ff6b6b')
    ax2.axhline(y=95, color='#4ecdc4', linestyle='--', alpha=0.5, label='TPR target (95%)')
    ax2.axhline(y=1, color='#ff6b6b', linestyle='--', alpha=0.5, label='FPR target (1%)')
    ax2.set_xlabel('Test Case')
    ax2.set_ylabel('Rate (%)')
    ax2.set_title('Detection Rates by Test Case')
    ax2.set_xticks(x)
    ax2.set_xticklabels(test_ids, rotation=45)
    ax2.legend(fontsize=8)
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('/Users/kunimitsu/Projects/Meguri_pre3/results/stage2_sybil_analysis.png',
                dpi=150, bbox_inches='tight')
    plt.show()
else:
    print('No Scenario A results to plot.')

## 4. Balance Distribution & Gini

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Gini coefficient by test case
ax1 = axes[0]
gini_vals = df_summary['Gini'].values
colors = ['#51cf66' if 0.2 <= g <= 0.3 else ('#ffec99' if g <= 0.5 else '#ff6b6b')
          for g in gini_vals]
ax1.bar(df_summary['Test ID'], gini_vals, color=colors)
ax1.axhline(y=0.2, color='green', linestyle='--', alpha=0.5, label='Target lower (0.2)')
ax1.axhline(y=0.3, color='green', linestyle='--', alpha=0.5, label='Target upper (0.3)')
ax1.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Warning (0.5)')
ax1.set_xlabel('Test Case')
ax1.set_ylabel('Gini Coefficient')
ax1.set_title('Balance Distribution Fairness')
ax1.legend(fontsize=8)
ax1.tick_params(axis='x', rotation=45)
ax1.grid(alpha=0.3)

# Bloom event distribution
ax2 = axes[1]
bloom_vals = df_summary['Bloom Events'].values
ax2.bar(df_summary['Test ID'], bloom_vals, color='#74b9ff')
ax2.set_xlabel('Test Case')
ax2.set_ylabel('Bloom Events')
ax2.set_title('Bloom Event Count by Test Case')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('/Users/kunimitsu/Projects/Meguri_pre3/results/stage2_fairness.png',
            dpi=150, bbox_inches='tight')
plt.show()

## 5. Per-Scenario Deep Dive

In [ ]:
# Load detailed scores for a specific test case
def analyze_test_case(test_id):
    """Load and analyze detailed scores for a test case."""
    csv_path = Path(f'/Users/kunimitsu/Projects/Meguri_pre3/results/scores/{test_id}_scores.csv')
    if not csv_path.exists():
        print(f'No score data for {test_id}')
        return None
    
    df = pd.read_csv(csv_path)
    
    # Final day analysis
    final_day = df['day'].max()
    final = df[df['day'] == final_day]
    honest = final[final['is_attacker'] == 0]
    attackers = final[final['is_attacker'] == 1]
    
    print(f'\n--- {test_id} (Day {final_day}) ---')
    print(f'Honest:    N={len(honest)}, S_mean={honest["S"].mean():.4f}, '
          f'F_rate={honest["F"].mean():.2%}, balance={honest["balance"].mean():.2f}')
    if len(attackers) > 0:
        print(f'Attackers: N={len(attackers)}, S_mean={attackers["S"].mean():.4f}, '
              f'F_rate={attackers["F"].mean():.2%}, balance={attackers["balance"].mean():.2f}')
    
    return df

# Analyze key scenarios
for r in all_results:
    analyze_test_case(r['test_id'])

## 6. Score Evolution Over 180 Days

In [ ]:
# Pick a medium-scale test case for evolution analysis
target_tid = None
for r in all_results:
    if r.get('scenario') == 'A' and r.get('attacker_count', 0) >= 5:
        target_tid = r['test_id']
        break

if target_tid:
    csv_path = Path(f'/Users/kunimitsu/Projects/Meguri_pre3/results/scores/{target_tid}_scores.csv')
    if csv_path.exists():
        df_evo = pd.read_csv(csv_path)
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle(f'Score Evolution: {target_tid}', fontsize=14, fontweight='bold')
        
        # Average S(v) over time: honest vs attackers
        honest_daily = df_evo[df_evo['is_attacker'] == 0].groupby('day')
        attacker_daily = df_evo[df_evo['is_attacker'] == 1].groupby('day')
        
        axes[0,0].plot(honest_daily['S'].mean(), label='Honest (mean)', color='#4ecdc4')
        axes[0,0].fill_between(honest_daily['S'].mean().index,
                              honest_daily['S'].mean() - honest_daily['S'].std(),
                              honest_daily['S'].mean() + honest_daily['S'].std(),
                              alpha=0.2, color='#4ecdc4')
        if len(attacker_daily) > 0:
            axes[0,0].plot(attacker_daily['S'].mean(), label='Attackers (mean)', color='#ff6b6b')
        axes[0,0].axhline(y=THETA, color='yellow', linestyle='--', alpha=0.5, label=f'θ={THETA}')
        axes[0,0].set_title('Integrated Score S(v)')
        axes[0,0].legend(fontsize=8)
        axes[0,0].grid(alpha=0.3)
        
        # Average balance over time
        axes[0,1].plot(honest_daily['balance'].mean(), label='Honest', color='#4ecdc4')
        if len(attacker_daily) > 0:
            axes[0,1].plot(attacker_daily['balance'].mean(), label='Attackers', color='#ff6b6b')
        axes[0,1].set_title('Average Balance')
        axes[0,1].legend(fontsize=8)
        axes[0,1].grid(alpha=0.3)
        
        # F(v) flag rate over time
        axes[1,0].plot(honest_daily['F'].mean() * 100, label='Honest FPR', color='#4ecdc4')
        if len(attacker_daily) > 0:
            axes[1,0].plot(attacker_daily['F'].mean() * 100, label='Attacker TPR', color='#ff6b6b')
        axes[1,0].axhline(y=1, color='yellow', linestyle='--', alpha=0.5, label='FPR target (1%)')
        axes[1,0].set_title('Anomaly Flag Rate (%)')
        axes[1,0].legend(fontsize=8)
        axes[1,0].grid(alpha=0.3)
        
        # Decay acceleration g(S) over time
        axes[1,1].plot(honest_daily['g_S'].mean(), label='Honest', color='#4ecdc4')
        if len(attacker_daily) > 0:
            axes[1,1].plot(attacker_daily['g_S'].mean(), label='Attackers', color='#ff6b6b')
        axes[1,1].axhline(y=1.0, color='yellow', linestyle='--', alpha=0.5, label='Normal (g=1)')
        axes[1,1].set_title('Decay Acceleration g(S)')
        axes[1,1].legend(fontsize=8)
        axes[1,1].grid(alpha=0.3)
        
        for ax in axes.flat:
            ax.set_xlabel('Day')
        
        plt.tight_layout()
        plt.savefig('/Users/kunimitsu/Projects/Meguri_pre3/results/stage2_evolution.png',
                    dpi=150, bbox_inches='tight')
        plt.show()
else:
    print('No suitable test case found for evolution analysis.')

## 7. Success Criteria Checklist

In [ ]:
print('\n' + '='*60)
print('PHASE 0 SUCCESS CRITERIA CHECKLIST')
print('='*60)

checks = [
    ('Sybil ROI(100) < 1/150', 'sybil_roi_pass', 'T004'),
    ('False Positive Rate < 1%', 'fpr_pass', None),
    ('κ, θ optimal values determined', None, 'Stage 3'),
    ('Bloom event interval determined', None, 'Stage 3'),
    ('Analysis cycle determined', None, 'Stage 3'),
    ('Grace Period determined', None, 'Stage 3'),
    ('Community detection algorithm selected', None, 'Louvain'),
    ('Distance axis necessity concluded', None, 'T010'),
]

for check_name, metric_key, note in checks:
    if metric_key:
        # Check across all results
        passes = all(r.get(metric_key, False) for r in all_results if r.get('attacker_count', 0) > 0)
        status = '☑' if passes else '☐'
    else:
        status = '☐'
    
    note_str = f' ({note})' if note else ''
    print(f'  {status} {check_name}{note_str}')

print()
print('Legend: ☑ = Achieved, ☐ = Pending (Stage 3 optimization needed)')
print('\nNext: Run notebooks/03_parameter_optimization.ipynb for DoE optimization')
print('='*60)